In [1]:
pip install pandas psycopg2-binary

Note: you may need to restart the kernel to use updated packages.


In [2]:
import pandas as pd
import psycopg2
from psycopg2.extensions import ISOLATION_LEVEL_AUTOCOMMIT
import sys

DB_NAME = "players"
DB_USER = "postgres"
DB_PASSWORD = "postgres"
DB_HOST = "localhost"
DB_PORT = "5432"
csv_file = "people.csv"

In [3]:
def create_database():
    """Create the database if it doesn't exist"""
    # Database connection parameters


    # Connect to PostgreSQL server
    try:
        conn = psycopg2.connect(
            user=DB_USER,
            password=DB_PASSWORD,
            host=DB_HOST,
            port=DB_PORT
        )
        conn.set_isolation_level(ISOLATION_LEVEL_AUTOCOMMIT)
        cursor = conn.cursor()

        # Create database
        cursor.execute(f"CREATE DATABASE {DB_NAME}")
        print(f"Database {DB_NAME} created successfully")

    except psycopg2.Error as e:
        if "already exists" in str(e):
            print(f"Database {DB_NAME} already exists")
        else:
            print(f"Error creating database: {e}")
            sys.exit(1)
    finally:
        if cursor:
            cursor.close()
        if conn:
            conn.close()

In [4]:
def create_table():
    """Create the players table"""
    conn = None
    try:
        conn = psycopg2.connect(
            dbname=DB_NAME,
            user=DB_USER,
            password=DB_PASSWORD,
            host=DB_HOST,
            port=DB_PORT
        )
        cursor = conn.cursor()

        # Create players table
        create_table_query = """
        CREATE TABLE IF NOT EXISTS players (
            identifier VARCHAR(100) PRIMARY KEY,
            name VARCHAR(200) NOT NULL,
            created_at TIMESTAMP DEFAULT CURRENT_TIMESTAMP
        )
        """
        cursor.execute(create_table_query)
        
        # Create index on name
        cursor.execute("CREATE INDEX IF NOT EXISTS idx_player_name ON players(name)")
        
        conn.commit()
        print("Players table created successfully")

    except psycopg2.Error as e:
        print(f"Error creating table: {e}")
        if conn:
            conn.rollback()
        sys.exit(1)
    finally:
        if cursor:
            cursor.close()
        if conn:
            conn.close()

In [ ]:
def load_data_from_csv(csv_file):
    """Load data from CSV file into the database"""
    try:
        # Read CSV file using pandas
        df = pd.read_csv(csv_file)
        
        # Take only first two columns
        players_df = df.iloc[:, [0, 1]]
        players_df.columns = ['identifier', 'name']
        
        # Remove duplicates based on identifier
        players_df = players_df.drop_duplicates(subset='identifier')
        
        # Connect to database
        conn = psycopg2.connect(
            dbname=DB_NAME,
            user=DB_USER,
            password=DB_PASSWORD,
            host=DB_HOST,
            port=DB_PORT
        )
        cursor = conn.cursor()

        # Insert data
        for _, row in players_df.iterrows():
            try:
                cursor.execute(
                    "INSERT INTO players (identifier, name) VALUES (%s, %s) ON CONFLICT (identifier) DO NOTHING",
                    (row['identifier'], row['name'])
                )
            except psycopg2.Error as e:
                print(f"Error inserting row {row['identifier']}: {e}")
                conn.rollback()
        
        conn.commit()
        print(f"Successfully loaded {len(players_df)} players into database")

    except pd.errors.EmptyDataError:
        print("The CSV file is empty")
    except FileNotFoundError:
        print(f"Could not find file: {csv_file}")
    except psycopg2.Error as e:
        print(f"Database error: {e}")
        if conn:
            conn.rollback()
    except Exception as e:
        print(f"An error occurred: {e}")
        if conn:
            conn.rollback()
    finally:
        if cursor:
            cursor.close()
        if conn:
            conn.close()

def main():
    # Specify your CSV file path

    
    print("Starting database creation process...")
    create_database()
    
    print("Creating players table...")
    create_table()
    
    print("Loading data from CSV...")
    load_data_from_csv(csv_file)
    
if __name__ == "__main__":
    main()

Starting database creation process...
Database players created successfully
Creating players table...
Players table created successfully
Loading data from CSV...
Successfully loaded 16063 players into database


In [7]:
#Update db from players csv if anything missing or newly added
# Read CSV file using pandas
df = pd.read_csv(csv_file)
print("Starting database creation process...")
create_database()

print("Creating players table...")
create_table()

# Connect to database
conn = psycopg2.connect(
    dbname=DB_NAME,
    user=DB_USER,
    password=DB_PASSWORD,
    host=DB_HOST,
    port=DB_PORT
)
cursor = conn.cursor()

# Take only first two columns
players_df = df.iloc[:, [0, 1]]
players_df.columns = ['identifier', 'name']

# Remove duplicates based on identifier
players_df = players_df.drop_duplicates(subset='identifier')

# Initialize counters
new_players_count = 0

# For each row in the CSV
for _, row in players_df.iterrows():
    # Check if player already exists using identifier
    cursor.execute('SELECT COUNT(*) FROM players WHERE identifier = %s', (row['identifier'],))
    exists = cursor.fetchone()[0]
    
    if not exists:
        # Insert new player
        cursor.execute(
                    "INSERT INTO players (identifier, name) VALUES (%s, %s) ON CONFLICT (identifier) DO NOTHING",
                    (row['identifier'], row['name'])
                )
        new_players_count += 1

# Commit changes and close connection
conn.commit()
cursor.close()
conn.close()

print(f"Import completed! Added {new_players_count} new players")



Starting database creation process...
Database players already exists
Creating players table...
Players table created successfully
Import completed! Added 0 new players
